# Decision Trees and Random Forests
### Prepared by: Mehdi Ghane
### Date: March 2020

* **Node**
    * Split for the value of a certain attribute
* **Edge**
    * Outcome of a split to next node
* **Root Node**
    * The node that performs the first split
* **Leaves**
    * Terminal nodes that predict the outcome

Entropy and Information Gain are the mathematical methods of choosing the best split. 
$$H(S) = -\sum_{i=1} p_i(S).log_2.p_i(S)$$

Maximizing the Information Gain is to try to choose a feature that best  split the data.

*Information gain = Entropy of parent node - average entropy of childeren nodes*

### Hyperparameters for Decision Trees
In order to create decision trees that will generalize to new problems well, we can tune a number of different aspects about the trees. We call the different aspects of a decision tree "hyperparameters". These are some of the most important hyperparameters used in decision trees:

**Maximum Depth**
The maximum depth of a decision tree is simply the largest possible length between the root to a leaf. A tree of maximum length k can have at most 2^k leaves.

**Minimum number of samples to split**
A node must have at least min_samples_split samples in order to be large enough to split. If a node has fewer samples than min_samples_split samples, it will not be split, and the splitting process stops. However, min_samples_split doesn't control the minimum size of leaves.

**Minimum number of samples per leaf**
When splitting a node, one could run into the problem of having 99 samples in one of them, and 1 on the other. This will not take us too far in our process, and would be a waste of resources and time. If we want to avoid this, we can set a minimum for the number of samples we allow on each leaf.

**HINT**
Large depth very often causes overfitting, since a tree that is too deep, can memorize the data. Small depth can result in a very simple model, which may cause underfitting.
Small minimum samples per split may result in a complicated, highly branched tree, which can mean the model has memorized the data, or in other words, overfit. Large minimum samples may result in the tree not having enough flexibility to get built, and may result in underfitting.

A model designed to have 100% accuracy on training data is unlikely to generalize well to new data. If you pick very large values for your parameters, the model will fit the training set very well, but may not generalize well. Try to find the smallest possible parameters that do the job—then the model will be more likely to generalize well.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# keep default values for hyperparameters
model_tree = DecisionTreeClassifier(
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1)

In [ ]:
model_tree.fit(X_train,y_train)
y_train_pred = model_tree.predict(X_train)
y_test_pred = model_tree.predict(X_test)

In [ ]:
print(classification_report(y_test,y_test_pred))
print('\n')
print(confusion_matrix(y_test,y_test_pred))
## Test data results in 0.79 accuracy.

In [ ]:
print(classification_report(y_train,y_train_pred))
print('\n')
print(confusion_matrix(y_train,y_train_pred))
## whereas the train set acheived 0.98 accuracy, a clear indication of overfitting issue

#### Improving model accuracy

It seems leaving hyperparmeters on deafult made overfitting which is a far difference between train and test accuracy. We will apply Grid Searching to find the optimum values for hyperparameters. The only downside is that Grid-searching can be extremely computationally expensive and may take a long time to run. Grid-Search will build a model on each parameters combination. It iterates through every parameter combination and stores a model for each combination. When constructing this class we must provide a dictionary of hyperparameters to evaluate in the param_grid argument. This is a map of the model parameter name and an array of values to attempt.

**More Insights**

* By default, accuracy is the score that is optimized, but other scores can be specified in the score argument of the GridSearchCV constructor.
* By default, the grid search will only take one thread. By setting the n_jobs argument in the GridSearchCV constructor to -1, the process can take use of all cores on our machine.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model_tree = DecisionTreeClassifier()

In [ ]:
# Build our criterion dictionary
criterion = ['gini', 'entropy']
splitter = ['best', 'random']
max_depth=[6, 7, 8]
min_samples_leaf=[ 4, 5, 6, 8]
min_samples_split = [8,9,10,12]
param_grid = dict(criterion=criterion,splitter=splitter,
                  max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                  min_samples_split=min_samples_split)

In [ ]:
grid = GridSearchCV(estimator=model_tree, param_grid=param_grid, n_jobs=-1, cv=3)

In [ ]:
import time
start_time = time.time()
grid_result = grid.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

In [ ]:
# Now that we found our optimal values of hyperparameters, we repeat the prediction using those values
model_tree = DecisionTreeClassifier(criterion='gini', splitter='random',
                                   max_depth=6, min_samples_leaf=6,
                                   min_samples_split=8)

In [ ]:
model_tree.fit(X_train, y_train)
y_train_pred = model_tree.predict(X_train)
y_test_pred = model_tree.predict(X_test)

In [ ]:
print(classification_report(y_train,y_train_pred))
print('\n')
print(confusion_matrix(y_train,y_train_pred))
print(classification_report(y_test,y_test_pred))
print('\n')
print(confusion_matrix(y_test,y_test_pred))

Running above code, an accuracy of 85% will be acheived for test set with a quite good resolution (not too small) values of hyperparameters.

**Random Forest** is a method to improve performance of single decision tree

To Apply Random forest we will build a **Boost Strap** of the sample trainning set